In [ ]:
# !pip install sqlalchemy

In [164]:
import sqlalchemy as db
import pandas as pd
import numpy
import re
import nltk
# from nltk.corpus import stopwords
# nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import SnowballStemmer
import spacy
NLP = spacy.load("en_core_web_sm")

In [39]:
avatar_id = '1'

In [134]:
sql_url = "mysql://root@localhost/toia"


engine = db.create_engine(sql_url)
connection = engine.connect()
metadata = db.MetaData()
videos = db.Table('video', metadata, autoload=True, autoload_with=engine)

avatar_kb = db.select([videos]).where(
    videos.columns.toia_id == avatar_id,
    videos.columns.private == 0,
    videos.columns.type.notin_(["filler", "exit"])
)

ResultProxy = connection.execute(avatar_kb)
ResultSet = ResultProxy.fetchall()


In [135]:
df_avatar = pd.DataFrame(ResultSet, 
             columns=[
                 'id_video', 
                 'type', 
                 'toia_id', 
                 'idx', 
                 'private', 
                 'question', 
                 'answer', 
                 'language', 
                 'likes', 
                 'views',
             ])

In [136]:
df_avatar

,id_video,type,toia_id,idx,private,question,answer,language,likes,views
0,0b2,answer,1,2,0,What is your favorite sport?,I love soccer!,en-US,2,5
1,a7c,answer,1,3,0,What do you study?,I study Computer Science.,en-US,10,34
2,ef1,answer,1,1,0,How are you?,I am fine thanks!,en-US,5,14
3,r21,answer,1,6,0,When is your birthday?,My birthday is in September.,en-US,4,20
4,r2a,answer,1,7,0,What do you do?,I am doing a Ph.D.,en-US,4,20
5,r2b,answer,1,8,0,How old are you?,I'm 35. Age last birthday,en-US,4,20
6,r2c,answer,1,9,0,Tell me something interesting?,I wrote a book about the ethics of artificial ...,en-US,4,20
7,r2e,greeting,1,18,0,hey,how's it going?,en-US,4,20
8,r2f,greeting,1,11,0,Hello,Hi!,en-US,4,20
9,r2h,no-answer,1,13,0,,"Sorry, I didn't get that",en-US,4,20


In [146]:
import spacy

nlp = spacy.load("en_core_web_sm")
docs = nlp.pipe(df_avatar['question'].values)
for doc in docs:
    print("--- next doc ---")
    print(doc)
    for token in doc:
        print(token.text, token.pos_, token.tag_)
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)

--- next doc ---
What is your favorite sport?
What PRON WP
is AUX VBZ
your PRON PRP$
favorite ADJ JJ
sport NOUN NN
? PUNCT .
--- next doc ---
What do you study?
What PRON WP
do AUX VBP
you PRON PRP
study VERB VB
? PUNCT .
--- next doc ---
How are you?
How ADV WRB
are AUX VBP
you PRON PRP
? PUNCT .
--- next doc ---
When is your birthday?
When ADV WRB
is AUX VBZ
your PRON PRP$
birthday NOUN NN
? PUNCT .
--- next doc ---
What do you do?
What PRON WP
do AUX VBP
you PRON PRP
do VERB VB
? PUNCT .
--- next doc ---
How old are you?
How ADV WRB
old ADJ JJ
are AUX VBP
you PRON PRP
? PUNCT .
--- next doc ---
Tell me something interesting?
Tell VERB VB
me PRON PRP
something PRON NN
interesting ADJ JJ
? PUNCT .
--- next doc ---
hey
hey INTJ UH
--- next doc ---
Hello
Hello INTJ UH
--- next doc ---

--- next doc ---
Howdy
Howdy INTJ UH
Howdy 0 5 PERSON
--- next doc ---
Do you like sushi?
Do AUX VBP
you PRON PRP
like VERB VB
sushi NOUN NN
? PUNCT .
--- next doc ---

--- next doc ---

--- next doc ---


In [149]:
doc = nlp("hey, hi!")

In [161]:
['INTJ', 'UH'] in [[token.pos_, token.tag_] for token in doc]

True

In [155]:
set([token.tag_ for token in doc])

{',', '.', 'UH'}

In [170]:
df_avatar[df_avatar['type'] == "greeting"].sample(n=1)['answer'].values[0]

"hey, I'm all right"

In [174]:
ps = SnowballStemmer('english')

def preprocess(text):
            # Stem and remove stopwords
            text = re.sub('[^a-zA-Z]', ' ', text)
            text = text.lower()
            text = text.split()
            text = [ps.stem(word) for word in text]  # if not word in set(stopwords.words('english'))]
            return ' '.join(text)


def toia_answer(query, dataset, k=1):
    doc = NLP(query)
    # if Greeting, greet
    if ['INTJ', 'UH'] in [[token.pos_, token.tag_] for token in doc]:
        answers = dataset[dataset['type'] == "greeting"].sample(n=1)
        return answers['answer'].values[0], answers['id_video'].values[0]
        
    querycorpus = []
    for i in range(0, len(df_avatar)):
        userquestion = preprocess(df_avatar['question'][i])
        querycorpus.append(userquestion)      

    # Creating the Bag of Words model with TFIDF and calc cosine_similarity
    vectorizer = CountVectorizer(decode_error="replace")
    vec_train = vectorizer.fit_transform(querycorpus) #this is needed to get the attribute vocabulary_
    training_vocabulary = vectorizer.vocabulary_
    transformer = TfidfTransformer()
    trainingvoc_vectorizer = CountVectorizer(decode_error="replace", vocabulary=training_vocabulary)
    tfidf_querycorpus = TfidfVectorizer().fit_transform(querycorpus)
    
    tfidf_userquestion = transformer.fit_transform(
        trainingvoc_vectorizer.fit_transform(
            numpy.array([
                preprocess(query)
            ]))) 
    cosine_similarities = cosine_similarity(tfidf_userquestion, tfidf_querycorpus)
    related_docs_indices = (-cosine_similarities[0]).argsort()
    sorted_freq = cosine_similarities[0][related_docs_indices]
    
    #note for this distance the problem we had befor with inf, we have now with 0. Again we decide
    #to make the prediction a bit random. This could be adjusted to remove any 0 distance and
    #pick the only ones left if any, and if none predict 1.
    
    if sum(sorted_freq)==0:
        answers = dataset[dataset['type'] == "no-answer"].sample(n=1)
        return answers['answer'].values[0], answers['id_video'].values[0]
    
    elif sorted_freq[k-1]!=sorted_freq[k] or sorted_freq[k-1]==sorted_freq[k]==0:
        selected = related_docs_indices[:k]
       
        return dataset.iloc[selected[0]]['answer'], dataset.iloc[selected[0]]['id_video']
    else:
        indeces = numpy.where(numpy.roll(sorted_freq,1)!=sorted_freq)
        selected = related_docs_indices[:indeces[0][indeces[0]>=k][0]]
    
        return dataset.iloc[selected[0]]['answer'], dataset.iloc[selected[0]]['id_video']

In [185]:
toia_answer("hola cachina", df_avatar)

("I can't answer this, sorryyyyy", 'r3y')